# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import sqlite3
import pandas as pd
import os
from sqlalchemy import create_engine

import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords

import numpy as np
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV

from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/mtx/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/mtx/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/mtx/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
#df = pd.read_sql("SELECT * FROM InsertTableName", engine)
df = pd.read_sql_table('MessageAndLabel',engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
df.iloc[511].to_dict()

{'aid_centers': 0,
 'aid_related': 0,
 'buildings': 0,
 'clothing': 0,
 'cold': 0,
 'death': 0,
 'direct_report': 1,
 'earthquake': 0,
 'electricity': 0,
 'fire': 0,
 'floods': 0,
 'food': 0,
 'genre': 'direct',
 'hospitals': 0,
 'id': 631,
 'infrastructure_related': 0,
 'medical_help': 0,
 'medical_products': 0,
 'message': 'There are so many people who came to city of Les Cayes, they have nowhere to stay.',
 'military': 0,
 'missing_people': 0,
 'money': 0,
 'offer': 0,
 'original': 'okay gen anpil ki soti nan kapital ki moun okay yo pa gen okenn kot',
 'other_aid': 0,
 'other_infrastructure': 0,
 'other_weather': 0,
 'refugees': 0,
 'related': 1,
 'request': 0,
 'search_and_rescue': 0,
 'security': 0,
 'shelter': 0,
 'shops': 0,
 'storm': 0,
 'tools': 0,
 'transport': 0,
 'water': 0,
 'weather_related': 0}

### 2. Write a tokenization function to process your text data

In [3]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
def pipeline_eval(Y_true,Y_pred):
    tmp_list = []
    for i,c in enumerate(Y_true.columns):
        accuracy = accuracy_score(Y_true[c], Y_pred[:,i])
        precision = precision_score(Y_true[c], Y_pred[:,i])
        recall = recall_score(Y_true[c], Y_pred[:,i])
        f1 = f1_score(Y_true[c], Y_pred[:,i])
        tmp_dict = {'feature':c,
                    'accuracy':accuracy,
                    'precision':precision,
                    'recall':recall,
                    'f1':f1
                   }
        tmp_list.append(tmp_dict)
        
    return pd.DataFrame(tmp_list).set_index('feature').sort_values('f1',ascending=False)

In [7]:
Y_train_pred = pipeline.predict(X_train)
eval_train = pipeline_eval(Y_train,Y_train_pred)
eval_train

,accuracy,f1,precision,recall
feature,,,,
related,0.992777,0.995299,0.993345,0.997261
aid_related,0.984734,0.981431,0.996583,0.966732
weather_related,0.987859,0.977976,0.996969,0.959694
earthquake,0.995799,0.977398,0.996627,0.958897
storm,0.994570,0.970056,0.997676,0.943925
food,0.993443,0.969967,0.998551,0.942974
request,0.987962,0.963896,0.999363,0.930861
shelter,0.992418,0.954960,0.999363,0.914336
direct_report,0.983146,0.954577,0.997691,0.915034


In [8]:
eval_train.describe().T[['mean','50%']]

,mean,50%
accuracy,0.992872,0.993443
f1,0.914573,0.914190
precision,0.998487,0.999316
recall,0.846945,0.843685


In [9]:
Y_test_pred = pipeline.predict(X_test)
eval0 = pipeline_eval(Y_test,Y_test_pred)
eval0.describe().T[['mean','50%']]

/Users/mtx/anaconda/envs/dlnd-tf-lab/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/mtx/anaconda/envs/dlnd-tf-lab/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


,mean,50%
accuracy,0.939652,0.956201
f1,0.215394,0.128440
precision,0.589990,0.727848
recall,0.158733,0.070707


### 6. Improve your model
Use grid search to find better parameters. 

In [15]:
pipeline.get_params()

{'clf': MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
             oob_score=False, random_state=None, verbose=0,
             warm_start=False),
            n_jobs=1),
 'clf__estimator': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
             oob_score=False, random_state=None, verbose=0,
             warm_start=False),
 'clf__estimator__bootstrap': True,
 'clf__estimator__class_we

In [18]:
parameters = {'vect__min_df': [1, 5],
              'clf__estimator__n_estimators':[10, 25], 
              'clf__estimator__max_depth':[3, 5],
              'clf__estimator__max_features': ['sqrt', 0.3]
             }

cv = GridSearchCV(pipeline, param_grid=parameters)

In [19]:
cv_model0 = cv.fit(X_train, Y_train)

In [20]:
cv_model0.cv_results_

{'mean_fit_time': array([ 10.15157406,   9.94672736,  11.73424967,  12.18626428,
         17.43695339,  15.37913744,  30.41821798,  27.30893532,
         10.1203297 ,  10.38176966,  12.33035501,  11.62182132,
         22.2277511 ,  18.79600739,  43.25027196,  35.03502162]),
 'mean_score_time': array([ 4.4843262 ,  4.43246698,  4.97545727,  4.95361463,  4.48345868,
         4.31890329,  4.97278333,  4.79757675,  4.40710425,  4.4726487 ,
         5.66926138,  4.3980213 ,  4.15895597,  4.10800934,  4.61076204,
         4.43337488]),
 'mean_test_score': array([ 0.19891399,  0.19788945,  0.19978485,  0.19942626,  0.2004508 ,
         0.19799191,  0.21689463,  0.21264279,  0.19799191,  0.19256186,
         0.19891399,  0.19840172,  0.21668972,  0.22247836,  0.2258081 ,
         0.22616669]),
 'mean_train_score': array([ 0.19909328,  0.19814559,  0.19981046,  0.19957994,  0.20862148,
         0.20547103,  0.22414323,  0.22045489,  0.19878592,  0.19412428,
         0.19927258,  0.19886276,  0.

In [21]:
cv_model0.best_params_

{'clf__estimator__max_depth': 5,
 'clf__estimator__max_features': 0.3,
 'clf__estimator__n_estimators': 25,
 'vect__min_df': 5}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [22]:
Y_test_pred = cv_model0.predict(X_test)
eval1 = pipeline_eval(Y_test,Y_test_pred)
eval1.describe().T[['mean','50%']]

/Users/mtx/anaconda/envs/dlnd-tf-lab/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/mtx/anaconda/envs/dlnd-tf-lab/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


,mean,50%
accuracy,0.943270,0.964500
f1,0.342871,0.350515
precision,0.650716,0.745763
recall,0.269460,0.228412


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [24]:
import pickle
pickle.dump(cv_model0, open('disaster_response_model.pkl', 'wb'))

In [27]:
from sklearn.externals import joblib
joblib.dump(cv_model0, 'disaster_response_model.m')

['disaster_response_model.m']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.